In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np 

print(tf.__version__)   # Tensorflow의 버전을 출력

mnist = keras.datasets.mnist

# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

In [ ]:
import copy

In [ ]:
plt.imshow(x_train[0],cmap=plt.cm.binary) #[1]은 x_train의 2번째 이미지 
plt.show()
print('target_label is :',y_train[0])

MNIST 데이터 셋 확인

데이터의 Shape, type, 차원 등 정보를 확인하겠다

In [ ]:
print('학습 손글씨 데이터 개수 :',len(x_train))
print('학습 손글씨 데이터 개수 :',len(y_train))
print('테스트 손글씨 데이터 개수 :',len(x_test))
print('테스트 손글씨 데이터 개수 :',len(y_test))

In [ ]:
x_train[0]

In [ ]:
x_train[0].shape

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
y_train_reshape = y_train.reshape((60000,-1))
y_test_reshape = y_test.reshape((10000,-1))

In [ ]:
y_train_reshape.shape

In [ ]:
y_test_reshape.shape

In [ ]:
y_train_reshape

In [ ]:
y_test_reshape

target label one-hot encoding

0~9까지 인식되는 타겟 데이터를 원 핫 인코딩 시키겠다 --> softmax에 넣기 위함

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse = False)
y_train_reshape_onehot = one_hot_encoder.fit_transform(y_train_reshape)
y_train_reshape_onehot

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse = False)
y_test_reshape_onehot = one_hot_encoder.fit_transform(y_test_reshape)
y_test_reshape_onehot

In [ ]:
y_train_reshape_onehot.shape

In [ ]:
y_test_reshape_onehot.shape

In [ ]:
y_test_reshape

28 x 28 데이터가 60000장 있는 것이다.
이를 input으로 넣어주어야한다.

- RGB가 아닌 Gray Scale이므로 Flatten은 불필요
- 255로 normalization 진행

In [ ]:
x_train_flatten = x_train.reshape(x_train.shape[0],-1)
x_test_flatten = x_test.reshape(x_test.shape[0],-1)

In [ ]:
x_train_flatten.shape

In [ ]:
x_test_flatten.shape

In [ ]:
x_train_flatten_normalize = x_train_flatten/255.
x_test_flatten_normalize = x_test_flatten/255.

모델링

- 기울기 계산을 통한 순전파, 역전파로 모델을 학습하겠다.

In [ ]:
def softmax(self, Z):
    expZ = np.exp(Z - np.max(Z))
    return expZ / expZ.sum(axis = 0, keepdims = True)

In [ ]:
def forward(self.X):
    store = {}

    A = X.T
    for l in range(self.L - 1):
        Z = self.parameters['W' + str(l+1)].dot(A) + self.parameters['b' + str(l+1)]
        A = self.sigmoid(Z)
        store['A' + str(l+1)] = A
        store['W' + str(l+1)] = self.parameters['W' + str(l+1)]
        store['Z' + str(l+1)] = Z
    
    Z = self.parameters['W' + str(self.L)].dot(A) + self.parameters['b' + str(L)]
    A = self.softmax(Z)
    store['A' + str(L)] = A
    store['W' + str(L)] = self.parameters['W' + str(L)]
    store['Z' + str(L)] = Z

    return A, store

In [ ]:
cost = -np.mean(Y * np.log(A.T + 1e-8))

In [ ]:
def backward(self,X,Y,store):

    derivatives = {}

    store['A0'] = X.T

    A = store['A' + str(self.L)]
    dZ = A - Y.T

    dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
    db = np.sum(dZ, axis = 1, keepdims = True) / self.n
    dAPrev = store['W' + str(self.L)].T.dot(dZ)

    derivatives['dW' + str(self.L)] = dW
    derivatives['db' + str(self.L)] = db

    for l in range(self.L - 1,0,-1):
        dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
        dW = dZ.dot(store['A' + str(l-1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        if l > 1:
            dAPrev = store['W' + str(l)].T.dot(dZ)

        derivatives["dW" + str(l)] = dW
		derivatives["db" + str(l)] = db

    return derivatives

In [ ]:
def pre_process_data(train_x, train_y, test_x, test_y):
    train_x = train_x / 255.
    test_x = test_x / 255.
 
    enc = OneHotEncoder(sparse=False, categories='auto')
    train_y = enc.fit_transform(train_y.reshape(len(train_y), -1))
 
    test_y = enc.transform(test_y.reshape(len(test_y), -1))
 
    return train_x, train_y, test_x, test_y

In [ ]:
def predict(self, X, Y):
	A, cache = self.forward(X)
	y_hat = np.argmax(A, axis=0)
	Y = np.argmax(Y, axis=1)
	accuracy = (y_hat == Y).mean()
	return accuracy * 100

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


mnist = keras.datasets.mnist

class ANN:
    def __init__(self, layers_size):
        self.layers_size = layers_size
        self.parameters = {}
        self.L = len(self.layers_size)
        self.n = 0
        self.costs = []
    
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z))
        return expZ / expZ.sum(axis=0, keepdims=True)
    
    def initialize_parameters(self):
        for l in range(1, len(self.layers_size)):
            self.parameters["W" + str(l)] = np.random.randn(self.layers_size[l], self.layers_size[l - 1]) / np.sqrt(self.layers_size[l - 1])
            self.parameters["b" + str(l)] = np.zeros((self.layers_size[l], 1))
    
    def forward(self, X):
        store = {}
 
        A = X.T
        for l in range(self.L - 1):
            Z = self.parameters["W" + str(l + 1)].dot(A) + self.parameters["b" + str(l + 1)]
            A = self.sigmoid(Z)
            store["A" + str(l + 1)] = A
            store["W" + str(l + 1)] = self.parameters["W" + str(l + 1)]
            store["Z" + str(l + 1)] = Z
 
        Z = self.parameters["W" + str(self.L)].dot(A) + self.parameters["b" + str(self.L)]
        A = self.softmax(Z)
        store["A" + str(self.L)] = A
        store["W" + str(self.L)] = self.parameters["W" + str(self.L)]
        store["Z" + str(self.L)] = Z
 
        return A, store
 
    def sigmoid_derivative(self, Z):
        s = 1 / (1 + np.exp(-Z))
        return s * (1 - s)

    def backward(self,X,Y,store):

        derivatives = {}

        store['A0'] = X.T

        A = store['A' + str(self.L)]
        dZ = A - Y.T

        dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        dAPrev = store['W' + str(self.L)].T.dot(dZ)

        derivatives['dW' + str(self.L)] = dW
        derivatives['db' + str(self.L)] = db

        for l in range(self.L - 1,0,-1):
            dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
            dW = dZ.dot(store['A' + str(l-1)].T) / self.n
            db = np.sum(dZ, axis = 1, keepdims = True) / self.n
            if l > 1:
                dAPrev = store['W' + str(l)].T.dot(dZ)
            derivatives["dW" + str(l)] = dW
            derivatives["db" + str(l)] = db

        return derivatives

    def fit(self, X, Y, learning_rate=0.01, n_iterations=300):
        self.n = X.shape[0]
 
        self.layers_size.insert(0, X.shape[1])
 
        self.initialize_parameters()
        for loop in range(n_iterations):
            A, store = self.forward(X)
            cost = -np.mean(Y * np.log(A.T+ 1e-8))
            derivatives = self.backward(X, Y, store)
 
            for l in range(1, self.L + 1):
                self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - learning_rate * derivatives["dW" + str(l)]
                self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - learning_rate * derivatives["db" + str(l)]
 
            if loop % 100 == 0:
                print("Cost: ", cost, "Train Accuracy:", self.predict(X, Y))
 
            if loop % 10 == 0:
                self.costs.append(cost)

    def predict(self, X, Y):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        Y = np.argmax(Y, axis=1)
        accuracy = (y_hat == Y).mean()
        return accuracy * 100
    
    def predict_output(self, X):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        return y_hat

    def plot_cost(self):
        plt.figure()
        plt.plot(np.arange(len(self.costs)), self.costs)
        plt.xlabel("epochs")
        plt.ylabel("cost")
        plt.show()

def pre_process_data(train_x, train_y, test_x, test_y):
    # Normalize
    train_x = train_x / 255.
    test_x = test_x / 255.
 
    enc = OneHotEncoder(sparse=False, categories='auto')
    train_y = enc.fit_transform(train_y.reshape(len(train_y), -1))
 
    test_y = enc.transform(test_y.reshape(len(test_y), -1))
    
    train_x_flatten = train_x.reshape(train_x.shape[0],-1)
    test_x_flatten = test_x.reshape(test_x.shape[0],-1)
    
    return train_x_flatten, train_y, test_x_flatten, test_y

if __name__ == '__main__':
    (train_x, train_y), (test_x, test_y) = mnist.load_data()
 
    train_x, train_y, test_x, test_y = pre_process_data(train_x, train_y, test_x, test_y)
 
    print("train_x's shape: " + str(train_x.shape))
    print("test_x's shape: " + str(test_x.shape))
    print("train_y's shape: " + str(train_y.shape))
    print("test_y's shape: " + str(test_y.shape))
 
 
    layers_dims = [50, 10]
 
    ann = ANN(layers_dims)
    ann.fit(train_x, train_y, learning_rate=0.1, n_iterations=300)
    print("Train Accuracy:", ann.predict(train_x, train_y))
    print("Test Accuracy:", ann.predict(test_x, test_y))
    ann.plot_cost()    

In [ ]:
ann

FGSM 구현

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3]

In [ ]:
# FGSM 공격 코드
def fgsm_attack(image, epsilon, data_grad):
    # data_grad 의 요소별 부호 값을 얻어옵니다
    sign_data_grad = np.sign(data_grad)
    # 입력 이미지의 각 픽셀에 sign_data_grad 를 적용해 작은 변화가 적용된 이미지를 생성합니다
    perturbed_image = image + epsilon*sign_data_grad
    # 값 범위를 [0,1]로 유지하기 위해 자르기(clipping)를 추가합니다
# perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # 작은 변화가 적용된 이미지를 리턴합니다
    return perturbed_image

In [ ]:
## predict_output에 한개씩 넣는 것부터 구현해야한다. (전체가 들어가는 것만 정상 작동함)

def test(model, test_x, test_y, epsilon, layers_dims):

    # 정확도 카운터
    correct = 0
    adv_examples = []

    # 테스트 셋의 모든 예제에 대해 루프를 돕니다
    
    for i in range(len(test_x)):
        data = test_x[i].reshape(1,784)
        target = test_y[i].reshape(1,10)
        
        A,store = model.forward(data)
        output = model.predict_output(data)
        
        #if output != np.argmax(target,axis = 1):
            #continue

        data_grad = model.backward(data,target,store)
        print(data_grad)

        #perturbed_data = fgsm_attack(data, epsilon, data_grad)

        #print(perturbed_data)

In [ ]:
## predict_output에 한개씩 넣는 것부터 구현해야한다. (전체가 들어가는 것만 정상 작동함)

def test(model, test_x, test_y, epsilon, layers_dims):


    data = test_x[0].reshape(1,784)
    target = test_y[0].reshape(1,10)
        
    A,store = model.forward(data)
    output = model.predict_output(data)
        
        #if output != np.argmax(target,axis = 1):
            #continue

    data_grad = model.backward(data,target,store)

    for data_grad.items():
        

    print(data_grad_array.shape)
    #perturbed_data = fgsm_attack(data, epsilon, data_grad)

    #print(perturbed_data)

In [ ]:
test(ann, test_x, test_y, epsilons, layers_dims)

In [ ]:
A = ann.predict_output(test_x0)
A

In [ ]:
np.sum(A)

In [ ]:
test_x0 = test_x[0].reshape(1,784)

In [ ]:
test_x0.shape

In [ ]:
def test(model, test_x, test_y, epsilon, layers_dims):

    # 정확도 카운터
    correct = 0
    adv_examples = []

    # 테스트 셋의 모든 예제에 대해 루프를 돕니다
    for data, target in test_x, test_y:

        # 데이터를 모델에 통과시킵니다
        output = model.predict_output(data)
        print(output)
        init_pred = output.max(1, keepdim=True)[1] # 로그 확률의 최대값을 가지는 인덱스를 얻습니다

        # 만약 초기 예측이 틀리면, 공격하지 않도록 하고 계속 진행합니다
        if init_pred.item() != target.item():
            continue

        # 손실을 계산합니다
        loss = F.nll_loss(output, target)

        # 모델의 변화도들을 전부 0으로 설정합니다
        model.zero_grad()

        # 후방 전달을 통해 모델의 변화도를 계산합니다
        loss.backward()

        # 변화도 값을 모읍니다
        data_grad = data.grad.data

        # FGSM 공격을 호출합니다
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # 작은 변화가 적용된 이미지에 대해 재분류합니다
        output = model(perturbed_data)

        # 올바른지 확인합니다
        final_pred = output.max(1, keepdim=True)[1] # 로그 확률의 최대값을 가지는 인덱스를 얻습니다
        if final_pred.item() == target.item():
            correct += 1
            # 0 엡실론 예제에 대해서 저장합니다
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # 추후 시각화를 위하 다른 예제들을 저장합니다
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples